In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import random_split
from torch.utils.data import Subset

In [4]:
import numpy as np
import time
import os
from PIL import Image

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [68]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms

class CustomDataset(Dataset):
    def __init__(self, root_dir, class_name, transform=None, limit=None):
        self.root_dir = root_dir
        self.class_name = class_name
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir)][:limit]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        return image, self.class_name

# 데이터셋 경로
dataset_path = '/kaggle/input/wall-classification'

# 데이터 변환 설정
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(300),
        transforms.RandomResizedCrop(256),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# 'bad' 폴더에서 1500장 이미지 가져오기
bad_dataset = CustomDataset(os.path.join(dataset_path, 'bad'), class_name=0, transform=data_transforms['train'], limit=1500)

# 'good', 'normal' 폴더에서 전체 이미지 가져오기
good_dataset = CustomDataset(os.path.join(dataset_path, 'good'), class_name=1, transform=data_transforms['train'])
normal_dataset = CustomDataset(os.path.join(dataset_path, 'normal'), class_name=2, transform=data_transforms['train'])

# 생성한 데이터셋 합치기
combined_dataset = Subset(bad_dataset, list(range(len(bad_dataset)))) + good_dataset + normal_dataset

# 데이터 로더 설정
batch_size = 32
train_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)

# 데이터셋 정보 출력
print("Number of images in combined_dataset:", len(combined_dataset))

Number of images in combined_dataset: 2869


In [69]:
from sklearn.model_selection import train_test_split

# 데이터셋을 train과 valid로 나누기 (8:2 비율)
dataset_size = len(combined_dataset)
train_size = int(0.8 * dataset_size)  # 전체 데이터셋 중 80%를 훈련 데이터로 사용

train_dataset, valid_dataset = train_test_split(combined_dataset, test_size=0.2, random_state=42)

# 데이터 로더 설정
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


In [75]:
model = models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 3)


# GPU 사용 가능하면 GPU로 전환
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


# 손실 함수와 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)


best_val_loss = float('inf')  # 초기값으로 무한대 설정
best_model_dir = '/kaggle/working'  # 모델을 저장할 경로



train_losses = []
valid_losses = []
valid_accuracies = []



# 모델 학습
num_epochs = 100
for epoch in range(num_epochs):
    start_time = time.time()  # 에포크 시작 시간
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        
    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)  # 훈련 손실 리스트에 추가
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_loader):.4f}')

    
    
    # Validation 성능 평가
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            
    avg_valid_loss = valid_loss / len(valid_loader)
    valid_losses.append(avg_valid_loss)  # 검증 손실 리스트에 추가
    valid_accuracy = 100 * correct / total
    valid_accuracies.append(valid_accuracy)  # 검증 정확도 리스트에 추가

            
        
    end_time = time.time()  # 에포크 종료 시간
    epoch_time = end_time - start_time  # 에포크 소요 시간

    scheduler.step(avg_valid_loss)  # 학습률 스케줄러 업데이트
    
    
    if valid_loss < best_val_loss:
        if epoch > 5:
            best_val_loss = valid_loss
            best_model_path = os.path.join(best_model_dir, f'normal_classification_{epoch+1}epoch.pth')
            torch.save(model.state_dict(), best_model_path)
            print(f'Saved best model with validation loss: {valid_loss/len(valid_loader):.4f} at epoch {epoch+1}')

    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Validation Loss: {valid_loss/len(valid_loader):.4f}, "
          f"Validation Accuracy: {(100 * correct / total):.2f}%")

Epoch [1/100], Training Loss: 1.0103
Epoch [1/100] - Validation Loss: 0.8806, Validation Accuracy: 61.67%
Epoch [2/100], Training Loss: 0.8898
Epoch [2/100] - Validation Loss: 0.9902, Validation Accuracy: 54.36%
Epoch [3/100], Training Loss: 0.8279
Epoch [3/100] - Validation Loss: 0.7640, Validation Accuracy: 64.63%
Epoch [4/100], Training Loss: 0.7806
Epoch [4/100] - Validation Loss: 0.7689, Validation Accuracy: 65.16%
Epoch [5/100], Training Loss: 0.7568
Epoch [5/100] - Validation Loss: 1.3253, Validation Accuracy: 48.43%
Epoch [6/100], Training Loss: 0.6680
Epoch [6/100] - Validation Loss: 1.3487, Validation Accuracy: 55.57%
Epoch [7/100], Training Loss: 0.6570
Saved best model with validation loss: 12.239794611930847 at epoch 7
Epoch [7/100] - Validation Loss: 0.6800, Validation Accuracy: 67.77%
Epoch [8/100], Training Loss: 0.5965
Epoch [8/100] - Validation Loss: 1.2457, Validation Accuracy: 58.54%
Epoch [9/100], Training Loss: 0.5556
Saved best model with validation loss: 11.8876

In [77]:
len(train_losses), len(valid_losses), len(valid_accuracies)

(100, 100, 100)

In [79]:
image_path = '/kaggle/input/wall-classification/good/S-210830_H_X_1_R_98451132009-1.jpg'
image = Image.open(image_path)
image = data_transforms['train'](image)
image = image.unsqueeze(0)  # 배치 차원 추가

# GPU 사용 가능하면 GPU로 전환
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image = image.to(device)
model.to(device)

# 예측 수행
with torch.no_grad():
    outputs = model(image)

# 예측 결과 확인
probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
predicted_class = torch.argmax(probabilities).item()

# 클래스 인덱스에 따른 클래스 이름 설정
class_names = ['bad', 'good', 'normal']

print("Predicted class:", class_names[predicted_class])
print("Class probabilities:", probabilities)

Predicted class: good
Class probabilities: tensor([7.7979e-02, 9.2154e-01, 4.8391e-04], device='cuda:0')
